In [1]:
from utils.music_utils import *
from pathlib import Path
import sqlite3
import logging
from typing import Tuple, Optional

In [2]:
DATASET = Path("MegaSet")
SQLITE_INSTANCE = "music.db"
pkl_files = list(DATASET.rglob("*.pkl"))
valid_files = [check_file_info(pkl_file) for pkl_file in pkl_files]
invalid_files = [pkl_file for pkl_file, valid in zip(pkl_files, valid_files) if not valid]
print(f"Valid files: {len(valid_files)}")
print(f"Invalid files: {len(invalid_files)}")

Valid files: 11637
Invalid files: 0


In [3]:
# pick random invalid file and print_info
import random
if invalid_files:
    random_invalid_file = random.choice(invalid_files)
    print_info(random_invalid_file)
else:
    print("No invalid files found")

No invalid files found


In [4]:
def get_data(mp3_file: Path) -> dict:
    pkl_path = mp3_file.with_suffix(".pkl")
    with open(pkl_path, "rb") as f:
        data = pickle.load(f)
    data.pop("predictions_87", None)
    data.pop("embedding_512", None)
    data["mp3_path"] = str(mp3_file)
    return data

In [5]:
rdm_mp3 = pick_random_mp3(DATASET)
rdm_mp3_data = get_data(rdm_mp3)

print("\n".join(f"{key}: {value}" for key, value in rdm_mp3_data.items()))

filename: 001 This Is.mp3
filepath: MegaSet/Jurassic 5/Jurassic 5 - Power In Numbers (2002)/001 This Is.mp3
folder: MegaSet/Jurassic 5/Jurassic 5 - Power In Numbers (2002)
filesize: 1.3
title: This Is
artist: Jurassic 5
album: Power in Numbers
year: 2002
tracknumber: 1
genre: Rap & Hip-Hop
top_5_genres: ['electronic', 'reggae', 'experimental', 'hiphop', 'alternative']
mp3_path: MegaSet/Jurassic 5/Jurassic 5 - Power In Numbers (2002)/001 This Is.mp3


In [6]:
def create_tables(self):
    try:
        with sqlite3.connect(self) as conn:
            c = conn.cursor()

            c.execute(
                """
                CREATE TABLE IF NOT EXISTS songs(
                    id INTEGER PRIMARY KEY,
                    filename TEXT NOT NULL,
                    filepath TEXT NOT NULL,
                    album_folder TEXT NOT NULL,
                    artist_folder TEXT NOT NULL,
                    filesize REAL NOT NULL,
                    title TEXT NOT NULL,
                    artist TEXT NOT NULL,
                    album TEXT NOT NULL,
                    year INTEGER,
                    tracknumber INTEGER,
                    genre TEXT,
                    top_5_genres TEXT
                )
                """
            )

    except sqlite3.Error as e:
        logging.error(f"An error occurred in create_tables: {e}")


create_tables(SQLITE_INSTANCE)

In [7]:
class Song:
    def __init__(
        self,
        filename: str,
        filepath: str,
        album_folder: str,
        artist_folder: str,
        filesize: float,
        title: str,
        artist: str,
        album: str,
        year: int,
        tracknumber: int,
        genre: str,
        top_5_genres: str,
    ):
        self.filename = filename
        self.filepath = filepath
        self.album_folder = album_folder
        self.artist_folder = artist_folder
        self.filesize = filesize
        self.title = title
        self.artist = artist
        self.album = album
        self.year = year if year else None
        self.tracknumber = tracknumber  if tracknumber else None
        self.genre = genre  if genre else None
        self.top_5_genres = (
            top_5_genres if type(top_5_genres) == str else ", ".join(top_5_genres)
        )

class DatabaseManager:
    def __init__(self, db_path: str):
        self.conn = sqlite3.connect(db_path, check_same_thread=False)

    def close(self):
        self.conn.close()

    def execute(self, query: str, params: Tuple = ()) -> None:
        try:
            with self.conn:
                self.conn.execute(query, params)
                self.conn.commit()
        except sqlite3.Error as e:
            logging.error(f"An error occurred: {e}")

    def fetchone(self, query: str, params: Tuple = ()) -> Optional[Tuple]:
        try:
            with self.conn:
                return self.conn.execute(query, params).fetchone()
        except sqlite3.Error as e:
            logging.error(f"An error occurred: {e}")

    def fetchall(self, query: str, params: Tuple = ()) -> Optional[Tuple]:
        try:
            with self.conn:
                return self.conn.execute(query, params).fetchall()
        except sqlite3.Error as e:
            logging.error(f"An error occurred: {e}")

    def count_songs(self) -> int:
        query = "SELECT COUNT(*) FROM songs"
        return self.fetchone(query)[0]

    def insert_song(self, song: Song) -> None:
        song_data = (
            song.filename,
            song.filepath,
            song.album_folder,
            song.artist_folder,
            song.filesize,
            song.title,
            song.artist,
            song.album,
            song.year,
            song.tracknumber,
            song.genre,
            song.top_5_genres,
        )
        query = "SELECT * FROM songs WHERE filename = ? AND filepath = ?"
        existing_song = self.fetchone(query, (song.filename, song.filepath))
        if existing_song:
            return
        query = """
            INSERT INTO songs 
            (filename, filepath, album_folder, artist_folder, filesize, title, artist, album, year, tracknumber, genre, top_5_genres) 
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        self.execute(query, song_data)

In [8]:
def pkl_to_dict(mp3_file: Path) -> Song:
    pkl_path = mp3_file.with_suffix(".pkl")
    with open(pkl_path, "rb") as f:
        data = pickle.load(f)
    data.pop("predictions_87", None)
    data.pop("embedding_512", None)
    data['artist_folder'] = data['filepath'].split("/")[1]
    song = Song(data["filename"], 
                data["filepath"], 
                data["folder"],
                data["artist_folder"],
                data["filesize"], 
                data["title"], 
                data["artist"], 
                data["album"],
                data["year"],
                data["tracknumber"],
                data["genre"],
                ",".join(data["top_5_genres"])
                )
    return song

In [9]:
db = DatabaseManager(SQLITE_INSTANCE)

song = pkl_to_dict(rdm_mp3)
db.insert_song(song)

In [10]:
db.fetchall("SELECT * FROM songs")[0]

(1,
 '001 This Is.mp3',
 'MegaSet/Jurassic 5/Jurassic 5 - Power In Numbers (2002)/001 This Is.mp3',
 'MegaSet/Jurassic 5/Jurassic 5 - Power In Numbers (2002)',
 'Jurassic 5',
 1.3,
 'This Is',
 'Jurassic 5',
 'Power in Numbers',
 2002,
 1,
 'Rap & Hip-Hop',
 'electronic,reggae,experimental,hiphop,alternative')

In [11]:
# loop over every file in DATASET and, read every .pkl into a Song object 
# and insert it into the database

for pkl_file in pkl_files:
    song = pkl_to_dict(pkl_file)
    db.insert_song(song)

# 27.7s

db.count_songs()

11637

In [12]:
# create a pandas dataframe from the database
import pandas as pd

query = "SELECT * FROM songs"
df = pd.read_sql_query(query, db.conn)

df.isna().sum()

id                  0
filename            0
filepath            0
album_folder        0
artist_folder       0
filesize            0
title               0
artist              0
album               0
year               28
tracknumber        46
genre            1262
top_5_genres        0
dtype: int64

In [13]:
import musicbrainzngs

musicbrainzngs.set_rate_limit(limit_or_interval=1.0, new_requests=10)
# If you plan to submit data, authenticate
# musicbrainzngs.auth("user", "password")

# (this step is required, as per the webservice access rules
# at http://wiki.musicbrainz.org/XML_Web_Service/Rate_Limiting )
musicbrainzngs.set_useragent("Music_Recommandation_Milvus", "0.1", "https://github.com/Hatchi-Kin/music_similarity")

# If you are connecting to a different server
# musicbrainzngs.set_hostname("beta.musicbrainz.org")

In [14]:
try:
    result = musicbrainzngs.search_release_groups("Moby", "Play")
    print("Request was successful")
except musicbrainzngs.NetworkError:
    print("Service is unavailable")
except musicbrainzngs.MusicBrainzError as e:
    print(f"An error occurred: {e}")

result['release-group-list'][0]

Request was successful


{'id': '6838bf7e-b70d-3299-b9eb-8f5ec90d5229',
 'type': 'Album',
 'ext:score': '100',
 'title': 'Moby',
 'first-release-date': '1992-07-27',
 'primary-type': 'Album',
 'artist-credit': [{'name': 'Moby',
   'artist': {'id': '8970d868-0723-483b-a75b-51088913d3d4',
    'name': 'Moby',
    'sort-name': 'Moby',
    'disambiguation': 'electronic musician Richard Melville Hall',
    'alias-list': [{'locale': 'ja',
      'sort-name': 'モービー',
      'type': 'Artist name',
      'alias': 'モービー'},
     {'locale': 'en',
      'sort-name': 'DJ Moby',
      'type': 'Artist name',
      'alias': 'DJ Moby'},
     {'locale': 'en',
      'sort-name': 'Moby',
      'type': 'Artist name',
      'primary': 'primary',
      'alias': 'Moby'}]}}],
 'release-list': [{'id': '2587659e-7469-4a63-896c-3f4571b0689b',
   'title': 'Moby',
   'status': 'Official'},
  {'id': '45dff336-26bb-4e9a-8a9d-9c40b7f480bc',
   'title': 'Moby',
   'status': 'Official'},
  {'id': '51dd83d1-0803-480f-8d8f-9efddf7a42d2',
   'title': 

In [15]:
def get_album_id(artist, album):
    result = musicbrainzngs.search_releases(artist=artist, release=album)
    if result['release-list']:
        # If there's a result, take the first one
        first_result = result['release-list'][0]
        return first_result['id']
    else:
        return None

# Replace 'Artist Name' and 'Album Name' with the artist and album you're interested in
print(get_album_id('Moby', 'Play'))

07833ad1-9638-3127-804a-e5a7d4e2ad21


In [16]:
def get_album_year_by_id(album_id):
        release_info = musicbrainzngs.get_release_by_id(album_id, includes=["release-groups"])
        date = release_info['release']['release-group']['first-release-date']
        # The date is usually in the format 'YYYY-MM-DD', so we take the first 4 characters to get the year
        year = date[:4]
        return year

# Replace 'Artist Name' and 'Album Name' with the artist and album you're interested in
print(get_album_year_by_id('07833ad1-9638-3127-804a-e5a7d4e2ad21'))

1999


In [17]:
# for every row in sqlite that has a None value for year,
# get the album_id and then the year and update the row in sqlite and the pkl file

errors = 0
year_none = db.fetchall("SELECT * FROM songs WHERE year IS NULL")
for row in year_none:
    album_id = get_album_id(row[6], row[5])
    if album_id:
        year = get_album_year_by_id(album_id)
        db.execute("UPDATE songs SET year = ? WHERE filename = ? AND filepath = ?", (year, row[1], row[2]))
        pkl_path = Path(row[2]).with_suffix(".pkl")
        with open(pkl_path, "rb") as f:
            data = pickle.load(f)
        data["year"] = year
        with open(pkl_path, "wb") as f:
            pickle.dump(data, f)
    else:
        errors += 1

# 9m 5.8s

print(f"Number of errors: {errors}")

Number of errors: 14


In [18]:
# count the number of rows with a None value or an empty string for year

db.fetchall("SELECT COUNT(*) FROM songs WHERE year IS NULL OR year = '' OR year = ' '")

[(28,)]

In [19]:
# Get the list of rows of sqlite table where genre is None
# read the .pkl file and get the first element of the top_5_genres list
# and update the row's value for genre in sqlite

genre_none = db.fetchall("SELECT * FROM songs WHERE genre IS NULL OR genre = '' OR genre = ' ' or genre = 'None'")
for row in genre_none:
    song = pkl_to_dict(Path(row[2]))
    genre = song.top_5_genres.split(",")[0]
    db.execute("UPDATE songs SET genre = ? WHERE filename = ?", (genre, row[1]))

In [20]:
# df.isna().sum() equivalent

result = db.fetchall("PRAGMA table_info(songs)")
columns = [column[1] for column in result]

# For each column, count the number of rows with NULL values
for column in columns:
    count = db.fetchone(f"SELECT COUNT(*) FROM songs WHERE {column} IS NULL")[0]
    print(f"{column}: {count}")

id: 0
filename: 0
filepath: 0
album_folder: 0
artist_folder: 0
filesize: 0
title: 0
artist: 0
album: 0
year: 14
tracknumber: 46
genre: 0
top_5_genres: 0


In [21]:
# Get the list of rows is sqlite that have a NULL value for tracknumber

tracknumber_none = db.fetchall("SELECT * FROM songs WHERE tracknumber IS NULL OR tracknumber = '' OR tracknumber = ' '")

# print the first 5 rows of the list
for row in tracknumber_none[:5]:
    # print(row[9])
    print(row[1])


Sinkane - Hold Tight.mp3
Chinese man records - Bouncefloor.mp3
Chinese man records - BB's Electrify BUGS.mp3
Chinese man records - Searching for Hiero.mp3
Chinese man records - Pudding à l'Arsenic.mp3


In [22]:
# read the value for filename and see if there is a str with two successive numbers
# for example '05 - Les nuits parisiennes.mp3'. if there is, 
# extract the number and update the value for tracknumber in sqlite and the .pkl file
    
import re

for row in tracknumber_none:
    filename = row[1]
    tracknumber = re.findall(r"\b\d{2}\b", filename)
    if tracknumber:
        tracknumber = int(tracknumber[0])
        db.execute("UPDATE songs SET tracknumber = ? WHERE filename = ?", (tracknumber, row[1]))
        pkl_path = Path(row[2]).with_suffix(".pkl")
        with open(pkl_path, "rb") as f:
            data = pickle.load(f)
        data["tracknumber"] = tracknumber
        with open(pkl_path, "wb") as f:
            pickle.dump(data, f)


In [23]:
# df.isna().sum() equivalent

result = db.fetchall("PRAGMA table_info(songs)")
columns = [column[1] for column in result]

# For each column, count the number of rows with NULL values
for column in columns:
    count = db.fetchone(f"SELECT COUNT(*) FROM songs WHERE {column} IS NULL")[0]
    print(f"{column}: {count}")

id: 0
filename: 0
filepath: 0
album_folder: 0
artist_folder: 0
filesize: 0
title: 0
artist: 0
album: 0
year: 14
tracknumber: 46
genre: 0
top_5_genres: 0


In [24]:
# print the rows where value for tracknumber is NULL
tracknumber_none = db.fetchall("SELECT * FROM songs WHERE tracknumber IS NULL OR tracknumber = '' OR tracknumber = ' '")
for row in tracknumber_none:
    print(f"{row[1]}   -----   {row[6]}")

Sinkane - Hold Tight.mp3   -----   Sinkane - Hold Tight
Chinese man records - Bouncefloor.mp3   -----   Chinese man records - Bouncefloor
Chinese man records - BB's Electrify BUGS.mp3   -----   Chinese man records - BB's Electrify BUGS
Chinese man records - Searching for Hiero.mp3   -----   Chinese man records - Searching for Hiero
Chinese man records - Pudding à l'Arsenic.mp3   -----   Pudding à l'Arsenic
Peter Tosh - Rastafari Is.mp3   -----   Rastafari Is
Peter Tosh - Mama Africa.mp3   -----   Peter Tosh - Mama Africa
Johnny Cash and June Carter - It Ain't Me, Babe.mp3   -----   Johnny Cash and June Carter - It Ain't Me, Babe
Johnny Cash & Bob Dylan - Complete Nashville Sessions - 1969.mp3   -----   AlbumWrap Album
Johnny Cash - Country Classics 1994 - Album.mp3   -----   All
Musica Country - Johnny Cash - Man In Black.mp3   -----   Man in Black
You always you (live) - Patrice.mp3   -----   You always you (live)
Jamie Cullum and Katie Melua - LoveCats (Brits 2004).mp3   -----   Jami

In [25]:
########### Unique artist

# for eah row in sqlite, read the filepath, extract the str between the first two / and compare to the value for artist
# if different, add the path to a list
db = DatabaseManager(SQLITE_INSTANCE)

artist_diff = []
for row in db.fetchall("SELECT * FROM songs"):
    filepath = row[2]
    artist = row[6]
    folder = filepath.split("/")[1]
    if folder != artist:
        artist_diff.append((filepath, artist))

In [26]:
for art in artist_diff[40:70]:
    print(art)

('MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/05 - La Colmena.mp3', 'La colmena')
('MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/02 - El Libertador.mp3', 'El libertador')
('MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/10 - El Tercero de la Foto.mp3', 'El tercero de la foto')
('MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/12 - Que Puedo Decir.mp3', 'Qué puedo decir')
('MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/03 - Crimen Sollicitationis.mp3', 'Crimen sollicitationis')
('MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/07 - El Imperio Caera.mp3', 'El imperio caerá')
('MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/04 - Fuego y Miedo.mp3', 'Fuego y miedo')
('MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/08 - Los Hijos Bastardos de la Globalizacion.mp3', 'Los hijos bastardos de la globalización')
('MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/01 - Ni Fu Ni Fa.mp3', 'Piste 1')
('MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/09 - Vandalo.mp3', 'Vándalo')
('MegaSet/Ska-P/Ska-P - 2008 -